In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [2]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'

In [3]:
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [4]:
df = df.iloc[:300]

In [5]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [6]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

/usr/local/python/3.10.13/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
answer_llm = df.iloc[0].answer_llm
v_llm = embedding_model.encode(answer_llm)
#v_llm

In [8]:
#answer_llm

In [9]:
def normalized_vector(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

In [10]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)
    
    return normalized_vector(v_llm).dot(normalized_vector(v_orig))

In [17]:
from rouge import Rouge
rouge_scorer = Rouge()
r = df.iloc[10]
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [18]:
def av_f_score(scores):
    return (scores['rouge-1']['f']+scores['rouge-2']['f']+scores['rouge-l']['f'])/3.

In [19]:
av_f_score(scores)

0.35490034990035496

In [28]:
def compute_rouge(record):
    score = rouge_scorer.get_scores(record['answer_llm'], record['answer_orig'])[0]
    return score

In [29]:
#all_scores_2

In [30]:
all_scores_2 = []
results_gpt4o = df.to_dict(orient='records')

for record in tqdm(results_gpt4o):
    score = compute_rouge(record)
    all_scores_2.append(score['rouge-2']['f'])

  0%|          | 0/300 [00:00<?, ?it/s]

In [31]:
df['rouge-2-f'] = all_scores_2 
df['rouge-2-f'].describe()

count    300.000000
mean       0.206965
std        0.153550
min        0.000000
25%        0.097809
50%        0.178671
75%        0.286181
max        0.739130
Name: rouge-2-f, dtype: float64